Import Libraries and Load Data

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Load the datasets
df_modif_reviews = pd.read_csv('modif_reviews.csv')
df_recipes_filtered = pd.read_csv('recipes_filtered_cleaned.csv')
fruits_df = pd.read_csv('fruits_data.csv')
vegetables_df = pd.read_csv('vegetables_data.csv')


Preprocess and Merge Data and and convert month in  numerical value

In [4]:
# Merge fruits and vegetables into one dataframe
seasonal_produce_df = pd.concat([fruits_df, vegetables_df])
seasonal_produce_df.columns = ['product_name', 'month', 'country']

# Convert month names to numeric values using a mapping dictionary
month_mapping = {
    'january': 1, 'february': 2, 'march': 3, 'april': 4,
    'may': 5, 'june': 6, 'july': 7, 'august': 8,
    'september': 9, 'october': 10, 'november': 11, 'december': 12
}
seasonal_produce_df['month'] = seasonal_produce_df['month'].str.lower().map(month_mapping)


Define Display Seasonal Produce Function

In [5]:
# function to display seasonal produce and get user selection
def display_seasonal_produce(country, month):
    month_numeric = month_mapping[month.lower()]
    seasonal_products = seasonal_produce_df[
        (seasonal_produce_df['country'].str.lower() == country.lower()) &
        (seasonal_produce_df['month'] == month_numeric)
    ]
    print(f"Seasonal Fruits and Vegetables for {country.capitalize()} in {month.capitalize()}:")
    print(seasonal_products[['product_name']].to_string(index=False))
    return seasonal_products['product_name'].str.lower().values


Define Create Features Function

In [6]:
# define a function to create features for clustering
def create_features(row, seasonal_product_names):
    ingredients = row['recipeingredientparts'].split(', ')
    seasonal_ingredients = [ingredient for ingredient in ingredients if ingredient in seasonal_product_names]
    return len(seasonal_ingredients)


Example Usage for Seasonal Produce

In [7]:
# example usage to define seasonal_product_names
selected_country = 'denmark'
selected_month = 'august'

seasonal_product_names = display_seasonal_produce(selected_country, selected_month)

# assuming the user selects available products
# for example, user selects 'cauliflower' and 'pear' from the list
user_selected_produce = ['cauliflower', 'pear']


Seasonal Fruits and Vegetables for Denmark in August:
        product_name
               apple
          blackberry
        blackcurrant
           blueberry
           muskmelon
                pear
                plum
  beech hat mushroom
            beetroot
         bell pepper
            broccoli
             cabbage
              carrot
         cauliflower
            celeriac
              celery
               chili
            cucumber
    emperor mushroom
      enoki mushroom
              fennel
    frillice lettuce
          green bean
    heart of lettuce
     iceberg lettuce
                leek
lollo bionda lettuce
 lollo rosso lettuce
            mushroom
    oak leaf lettuce
               onion
     oyster mushroom
            pak choi
             parsnip
                 pea
 portobello mushroom
             pumpkin
         red cabbage
             rhubarb
     romaine lettuce
             salsify
             shallot
             spinach
              squash
 

 Apply Create Features Function

In [8]:
# apply the create_features function to add a new column
df_recipes_filtered['num_seasonal_ingredients'] = df_recipes_filtered.apply(create_features, axis=1, args=(user_selected_produce,))


Select Features for Clustering and Standardize

In [9]:
# select features for clustering
features = ['aggregatedrating', 'reviewcount', 'cooktime', 'preptime', 'totaltime', 'num_seasonal_ingredients']
X = df_recipes_filtered[features]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


Fit K-means Model and Add Cluster Information

In [10]:
# Fit the K-means model with the optimal number of clusters (4 clusters) and set n_init explicitly
kmeans = KMeans(n_clusters=4, n_init=10, random_state=42)
df_recipes_filtered['cluster'] = kmeans.fit_predict(X_scaled)


Define Recommendation Function

In [11]:
# Recommendation function
def recommend_recipes(selected_country, selected_month, selected_produce):
    # Convert month name to numeric value
    selected_month_numeric = month_mapping[selected_month.lower()]
    
    # Filter seasonal produce based on user selection
    seasonal_products = seasonal_produce_df[
        (seasonal_produce_df['country'] == selected_country) &
        (seasonal_produce_df['month'] == selected_month_numeric)
    ]
    
    # Get the list of seasonal products
    seasonal_product_names = seasonal_products['product_name'].str.lower().values
    
    # Find recipes with seasonal products and cluster info
    def ingredient_match(ingredients, products):
        return any(product in ingredients for product in products)
    
    matching_recipes = df_recipes_filtered[
        df_recipes_filtered['recipeingredientparts'].apply(ingredient_match, args=(selected_produce,))
    ]
    
    if matching_recipes.empty:
        return "No matching recipes found."
    else:
        # Return the recipes along with their details and cluster information
        return matching_recipes[['recipeid', 'name', 'aggregatedrating', 'reviewcount', 'cluster', 'recipeingredientparts', 'recipeinstructions']]


Usage of Recommendation Function

In [12]:
# Example usage
recommended_recipes = recommend_recipes(selected_country, selected_month, user_selected_produce)
display(recommended_recipes)


,recipeid,name,aggregatedrating,reviewcount,cluster,recipeingredientparts,recipeinstructions
209,264,trifle,4.0,3.0,0,egg white substitute granulated sugar soymilk ...,preheat the oven to 350 degrees beat the egg w...
213,270,spiced pear butter,5.0,9.0,0,pears innamon sticks allspice loves sugar,combine pears and apple juice in a large dutch...
216,273,winter pear butter,5.0,3.0,0,pears sugar brandy water,peel quarter and ore the pears hop in 12inch p...
234,292,curried vegetables,0.0,0.0,1,cauliflower arrots elery onion mayonnaise salt...,cook the auliflower arrots elery and onion in ...
285,347,roasted tomato and mint salsa,0.0,0.0,1,roma tomatoes garlic love lime juice extra vir...,with a omal or black iron skillet over mediumh...
...,...,...,...,...,...,...,...
521152,540793,vegetable masala,0.0,0.0,1,cauliflower potatoes whole umin seed onion gro...,cut the auliflower into florets about 25 mm ac...
521201,540844,grilled brie ham amp pear sandwich,0.0,0.0,1,bosc pear pomegranate molasses black pepper bu...,assemble sandwich by placing slices of brie fo...
521246,540892,vegetarian cauliflower cake,0.0,0.0,1,cauliflower kosher salt red onion olive oil fr...,preheat the oven to 400ºf200ºc place the aulif...
521463,541115,spicy roasted vegetable soup with leftover gri...,0.0,0.0,1,sweet potato baby potatoes turnip parsnip arro...,1 preheat oven to 425 degrees f line 2 large s...
